In [2]:
%load_ext autoreload 
%autoreload 2
import process as p
import pandas as pd
import dmc
df = p.processed_data()
start, end, split = pd.Timestamp('2014-1-1'), pd.Timestamp('2014-12-31'), pd.Timestamp('2014-10-1')
df.orderDate = pd.to_datetime(df.orderDate)
mask = (df.orderDate >= start) & (df.orderDate <= end)
df_full = df[mask]
X, Y, fts = dmc.transformation.transform_preserving_header(df_full, scaler=dmc.transformation.scale_features, binary_target=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/bpm1/BigData/run-dmc/venv/lib/python3.4/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [4]:
n = 50
impact_map = dict()
feature_set = set(fts)
for feature in fts:
    impact_map[feature] = 0.

for j in range(n):
    print(j)
    df = p.shuffle(df)
    start, end, split = pd.Timestamp('2014-1-1'), pd.Timestamp('2014-12-31'), pd.Timestamp('2014-10-1')
    df.orderDate = pd.to_datetime(df.orderDate)
    mask = (df.orderDate >= start) & (df.orderDate <= end)
    df_full = df[mask]
    te_size = 10000
    tr_size = 10000
    X, Y, fts = dmc.transformation.transform_preserving_header(df_full, scaler=dmc.transformation.scale_features, binary_target=True)
    train = X[:tr_size], Y[:tr_size]
    test = X[tr_size:tr_size+te_size], Y[tr_size:tr_size+te_size]

    res_tree = dmc.evaluation.evaluate_features_leaving_one_out(train[0], train[1], test[0], test[1], fts, dmc.classifiers.DecisionTree)
    for feature in feature_set:
            impact_map[feature] = impact_map[feature] + res_tree.decrement[feature]
    print("After run #" + str(j) + ": ")
    print(impact_map)
            
for key, value in impact_map.items():
    impact_map[key] = value / n

import operator
sorted_impact_list = sorted(impact_map.items(), key=operator.itemgetter(1))
print(sorted_impact_list)

0


/home/bpm1/BigData/run-dmc/venv/lib/python3.4/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


After run #0: 
{'t_order_boughtArticleTypeCount': 0.00070000000000003393, 't_article_availableSizes': 0.00050000000000005596, 't_sizeCodeNumerized': -0.0020000000000000018, 'surplusArticleQuantity': -0.0013999999999999568, 't_order_priceStd_A': 0.0035000000000000586, 'quantity': -9.9999999999988987e-05, 'orderSeason': -0.0015999999999999348, 'orderDayOfYear': -0.0024999999999999467, 'orderWeekDay': 0.0019000000000000128, 't_isChristmas': 0.00019999999999997797, 't_order_sameArticlesCount': -0.00089999999999990088, 't_voucher_is15PercentVoucher': 0.0018000000000000238, 'orderDay': -0.00039999999999995595, 't_isWeekend_A': 0.0, 'sizeReturnProb': 0.0041999999999999815, 'customerAvgUnisize': -0.0020999999999999908, 't_paymentWithFee_A': 0.00019999999999997797, 'productGroupReturnProb': 0.0072000000000000952, 't_article_boughtCountGlobal': -0.00059999999999993392, 't_reducedPaymentMethod_A': -0.00019999999999997797, 'deviceID': 0.0017000000000000348, 'orderTotalDay': 0.0010000000000001119, 